### 508p

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc

In [4]:
inf = pd.read_excel("01_시군구별 제1권-서울 인천 경기 강원_2019년.xlsx",sheet_name='82p인구')


# 1. 3번 행을 칼럼명으로 변경
inf.columns = inf.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf = inf.drop(index=[0, 1, 2, 3, 4, 5, 6, 7])

# 3. 46, 47, 48번 행 삭제
inf = inf.drop(index=[57, 58, 59])

# 인덱스 리셋
inf = inf.reset_index(drop=True)

inf.iloc[:, [2, 3]] = inf.iloc[:, [3, 2]].values
inf.iloc[:, [6, 7]] = inf.iloc[:, [7, 6]].values

inf = inf.loc[:, ~inf.columns.isna()]

inf = inf.drop(columns = ["G. & T.", "Division"])

inf.columns = [col.split('\n')[0] for col in inf.columns]
inf.columns = [col.split('  C.E')[0] for col in inf.columns]
inf.columns = [col.split('  S.E.')[0] for col in inf.columns]
inf.columns = [col.split('    Medical Aid')[0] for col in inf.columns]

# 창원시 관련 행 필터링 및 합계 계산
suwon_gu = ["수원시장안구           ", "수원시권선구           ", "수원시팔달구           ", "수원시영통구           "]
suwon_data = inf[inf['구  분'].isin(suwon_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
suwon_row = pd.DataFrame([['수원시'] + suwon_data.tolist()], columns=inf.columns)

# 창원시 관련 행 필터링 및 합계 계산
sungnam_gu = ["성남시수정구           ", "성남시중원구           ", "성남시분당구           "]
sungnam_data = inf[inf['구  분'].isin(sungnam_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
sungnam_row = pd.DataFrame([['성남시'] + sungnam_data.tolist()], columns=inf.columns)

# 창원시 관련 행 필터링 및 합계 계산
anyang_gu = ["안양시만안구           ", "안양시동안구           "]
anyang_data = inf[inf['구  분'].isin(anyang_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
anyang_row = pd.DataFrame([['안양시'] + anyang_data.tolist()], columns=inf.columns)

# 창원시 행 추가 및 기존 구 삭제
inf = pd.concat([inf[~inf['구  분'].isin(suwon_gu)], suwon_row], ignore_index=True)
inf = pd.concat([inf[~inf['구  분'].isin(sungnam_gu)], sungnam_row], ignore_index=True)
inf = pd.concat([inf[~inf['구  분'].isin(anyang_gu)], anyang_row], ignore_index=True)
inf = inf.rename(columns={'구  분': '지역'})

inf['지역'] = inf['지역'] + ' ' + inf.groupby('지역').cumcount().replace(0, '').astype(str)

# inf.columns = ['구  분', '진료실인원수(관외)', '입내원일수(관외)', '요양급여일수(관외)', '진료비(관외)', '급여비(관외)']
inf

,지역,의료보장,근 로 자,공ㆍ교,지 역,의료급여 1종,의료급여 2종
0,서울,9933336,6410576,576211,2695880,179257,71412
1,종로구,157094,92512,11069,49231,3442,840
2,중구,138071,84314,5503,44532,2967,755
3,용산구,231921,143113,14686,68331,4416,1375
4,성동구,305714,201231,14947,82873,4507,2156
5,광진구,359122,231676,17531,103182,4664,2069
6,동대문구,348727,215560,17378,105242,7849,2698
7,중랑구,397458,242088,15248,126078,10248,3796
8,성북구,443118,279766,26575,125710,7630,3437
9,강북구,312878,184095,11894,102666,9914,4309


In [5]:
inf2 = pd.read_excel("01_시군구별 제1권-서울 인천 경기 강원_2019년.xlsx",sheet_name='84p')


# 1. 3번 행을 칼럼명으로 변경
inf2.columns = inf2.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf2 = inf2.drop(index=[0, 1, 2, 3, 4, 5, 6, 7])

# 3. 46, 47, 48번 행 삭제
inf2 = inf2.drop(index=[58, 59, 60])

# 인덱스 리셋
inf2 = inf2.reset_index(drop=True)

inf2.iloc[:, [2, 3]] = inf2.iloc[:, [3, 2]].values
inf2.iloc[:, [6, 7]] = inf2.iloc[:, [7, 6]].values

inf2 = inf2.loc[:, ~inf2.columns.isna()]

inf2 = inf2.drop(columns = ["G. & T.", "Division"])

inf2.columns = [col.split('\n')[0] for col in inf2.columns]
inf2.columns = [col.split('  C.E')[0] for col in inf2.columns]
inf2.columns = [col.split('  S.E.')[0] for col in inf2.columns]
inf2.columns = [col.split('    Medical Aid')[0] for col in inf2.columns]

# 창원시 관련 행 필터링 및 합계 계산
ansan_gu = ["안산시상록구           ", "안산시단원구           "]
ansan_data = inf2[inf2['구  분'].isin(ansan_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
ansan_row = pd.DataFrame([['안산시'] + ansan_data.tolist()], columns=inf2.columns)

# 창원시 관련 행 필터링 및 합계 계산
goyang_gu = ["고양시덕양구           ", "고양시일산동구", "고양시일산서구"]
goyang_data = inf2[inf2['구  분'].isin(goyang_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
goyang_row = pd.DataFrame([['고양시'] + goyang_data.tolist()], columns=inf2.columns)

# 창원시 관련 행 필터링 및 합계 계산
yongin_gu = ["용인시처인구", "용인시기흥구", "용인시수지구"]
yongin_data = inf2[inf2['구  분'].isin(yongin_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
yongin_row = pd.DataFrame([['용인시'] + yongin_data.tolist()], columns=inf2.columns)

# 창원시 행 추가 및 기존 구 삭제
inf2 = pd.concat([inf2[~inf2['구  분'].isin(ansan_gu)], ansan_row], ignore_index=True)
inf2 = pd.concat([inf2[~inf2['구  분'].isin(goyang_gu)], goyang_row], ignore_index=True)
inf2 = pd.concat([inf2[~inf2['구  분'].isin(yongin_gu)], yongin_row], ignore_index=True)
inf2 = inf2.rename(columns={'구  분': '지역'})

inf2['지역'] = inf2['지역'] + ' ' + inf2.groupby('지역').cumcount().replace(0, '').astype(str)

# inf2.columns = ['구  분', '진료실인원수(관외)', '입내원일수(관외)', '요양급여일수(관외)', '진료비(관외)', '급여비(관외)']
inf2

,지역,의료보장,근 로 자,공ㆍ교,지 역,의료급여 1종,의료급여 2종
0,광명시,322688,221135,17007,78637,4510,1399
1,평택시,533575,346548,31711,145060,7848,2408
2,동두천시,95374,53597,7300,30406,3474,597
3,과천시,58858,39229,6839,12014,549,227
4,구리시,200048,128901,12067,54418,3397,1265
5,남양주시,709312,446371,40708,209138,10157,2938
6,오산시,236644,162495,8196,62643,2617,693
7,시흥시,512586,346199,17673,140344,6098,2272
8,군포시,283322,196994,17156,64175,3904,1093
9,의왕시,162992,113622,11292,36099,1513,466


In [6]:
inf3 = pd.read_excel("01_시군구별 제1권-서울 인천 경기 강원_2019년.xlsx",sheet_name='86p연령')


# 1. 3번 행을 칼럼명으로 변경
inf3.columns = inf3.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf3 = inf3.drop(index=[0, 1, 2, 3, 4, 5, 6, 7])

# 3. 46, 47, 48번 행 삭제
inf3 = inf3.drop(index=[57, 58])

# 인덱스 리셋
inf3 = inf3.reset_index(drop=True)

inf3 = inf3.drop(columns = [ "Division"])

inf3.columns = [col.split('\n')[0] for col in inf3.columns]

# 창원시 관련 행 필터링 및 합계 계산
suwon_gu = ["수원시장안구           ", "수원시권선구           ", "수원시팔달구           ", "수원시영통구           "]
suwon_data = inf3[inf3['구  분'].isin(suwon_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
suwon_row = pd.DataFrame([['수원시'] + suwon_data.tolist()], columns=inf3.columns)

# 창원시 관련 행 필터링 및 합계 계산
sungnam_gu = ["성남시수정구           ", "성남시중원구           ", "성남시분당구           "]
sungnam_data = inf3[inf3['구  분'].isin(sungnam_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
sungnam_row = pd.DataFrame([['성남시'] + sungnam_data.tolist()], columns=inf3.columns)

# 창원시 관련 행 필터링 및 합계 계산
anyang_gu = ["안양시만안구           ", "안양시동안구           "]
anyang_data = inf3[inf3['구  분'].isin(anyang_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
anyang_row = pd.DataFrame([['안양시'] + anyang_data.tolist()], columns=inf3.columns)

# 창원시 행 추가 및 기존 구 삭제
inf3 = pd.concat([inf3[~inf3['구  분'].isin(suwon_gu)], suwon_row], ignore_index=True)
inf3 = pd.concat([inf3[~inf3['구  분'].isin(sungnam_gu)], sungnam_row], ignore_index=True)
inf3 = pd.concat([inf3[~inf3['구  분'].isin(anyang_gu)], anyang_row], ignore_index=True)
inf3 = inf3.rename(columns={'구  분': '지역'})
inf3['지역'] = inf3['지역'] + ' ' + inf3.groupby('지역').cumcount().replace(0, '').astype(str)
# inf3.at[2, '지역'] = 중구
# inf3.columns = ['구  분', '진료실인원수(관외)', '입내원일수(관외)', '요양급여일수(관외)', '진료비(관외)', '급여비(관외)']
inf3

,지역,계,0-4세,5-9세,10-14세,15-19세,20-24세,25-29세,30-34세,35-39세,40-44세,45-49세,50-54세,55-59세,60-64세,65-69세,70-74세,75-79세,80세 이상
0,서울,9933336,307840,366494,372199,449004,647850,838952,760167,809190,743110,857604,801587,793103,722805,496505,385309,291243,290374
1,종로구,157094,3727,4828,5223,6622,11235,13645,11438,11201,10651,13664,13400,13590,11714,7869,6447,5635,6205
2,중구,138071,4191,4012,3316,4276,8284,12511,12551,12056,9687,10792,11236,11886,10797,7315,5593,4586,4982
3,용산구,231921,7071,7480,7365,9120,13895,20197,19906,20415,17175,20007,18878,18288,16335,10955,8653,7483,8698
4,성동구,305714,11377,10926,9618,11859,20526,26383,25431,27367,22857,25349,24720,24733,21596,14117,11068,8983,8804
5,광진구,359122,10168,11964,12394,15271,26366,36353,30803,29279,25876,29915,29279,28737,25341,16401,12359,9366,9250
6,동대문구,348727,10675,11786,11334,14002,25641,30167,25182,27656,24966,28028,27397,27862,26346,18657,15074,12278,11676
7,중랑구,397458,11909,13028,12543,15999,24636,31943,28937,31179,27268,32224,34510,36449,32743,22152,17042,12930,11966
8,성북구,443118,13820,17463,17564,21508,32163,34657,29281,33909,32704,40155,35931,34005,31100,21584,17738,14811,14725
9,강북구,312878,7734,9769,10293,13319,20371,23230,19269,22137,21744,27033,26378,27125,25369,18533,15918,13109,11547


In [7]:
inf4 = pd.read_excel("01_시군구별 제1권-서울 인천 경기 강원_2019년.xlsx",sheet_name='88p')


# 1. 3번 행을 칼럼명으로 변경
inf4.columns = inf4.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf4 = inf4.drop(index=[0, 1, 2, 3, 4, 5, 6, 7])

# 3. 46, 47, 48번 행 삭제
inf4 = inf4.drop(index=[58, 59])

# 인덱스 리셋
inf4 = inf4.reset_index(drop=True)

inf4 = inf4.drop(columns = [ "Division"])

inf4.columns = [col.split('\n')[0] for col in inf4.columns]

# 창원시 관련 행 필터링 및 합계 계산
ansan_gu = ["안산시상록구           ", "안산시단원구           "]
ansan_data = inf4[inf4['구  분'].isin(ansan_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
ansan_row = pd.DataFrame([['안산시'] + ansan_data.tolist()], columns=inf4.columns)

# 창원시 관련 행 필터링 및 합계 계산
goyang_gu = ["고양시덕양구           ", "고양시일산동구", "고양시일산서구"]
goyang_data = inf4[inf4['구  분'].isin(goyang_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
goyang_row = pd.DataFrame([['고양시'] + goyang_data.tolist()], columns=inf4.columns)

# 창원시 관련 행 필터링 및 합계 계산
yongin_gu = ["용인시처인구", "용인시기흥구", "용인시수지구"]
yongin_data = inf4[inf4['구  분'].isin(yongin_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
yongin_row = pd.DataFrame([['용인시'] + yongin_data.tolist()], columns=inf4.columns)

# 창원시 행 추가 및 기존 구 삭제
inf4 = pd.concat([inf4[~inf4['구  분'].isin(ansan_gu)], ansan_row], ignore_index=True)
inf4 = pd.concat([inf4[~inf4['구  분'].isin(goyang_gu)], goyang_row], ignore_index=True)
inf4 = pd.concat([inf4[~inf4['구  분'].isin(yongin_gu)], yongin_row], ignore_index=True)
inf4 = inf4.rename(columns={'구  분': '지역'})


inf4 = inf4.rename(columns={'구  분': '지역'})
# inf4.columns = ['구  분', '진료실인원수(관외)', '입내원일수(관외)', '요양급여일수(관외)', '진료비(관외)', '급여비(관외)']
inf4

,지역,계,0-4세,5-9세,10-14세,15-19세,20-24세,25-29세,30-34세,35-39세,40-44세,45-49세,50-54세,55-59세,60-64세,65-69세,70-74세,75-79세,80세 이상
0,광명시,322688,11294,14752,15430,17683,20137,21406,20859,25764,26106,30226,27817,26491,23317,14828,10338,7802,8438
1,평택시,533575,22615,26764,25671,27702,33787,39131,38067,45742,44641,49234,45468,41457,31597,19512,14831,12566,14790
2,동두천시,95374,2866,4120,4203,4794,5821,5823,4871,6221,6691,8305,8170,8138,7414,4897,4460,4109,4471
3,과천시,58858,1953,2573,2866,3471,4020,4018,3797,4186,3985,5421,5408,5108,4070,2517,1862,1548,2055
4,구리시,200048,7328,9059,8705,10392,13664,14132,12060,15885,15539,18245,18300,17768,14106,8526,6293,4987,5059
5,남양주시,709312,28370,39298,38316,39390,43585,41127,37659,57196,62029,68313,59677,55160,46283,30837,24021,18752,19299
6,오산시,236644,11495,14367,12753,12370,14248,17246,18449,24530,22384,21606,18407,15939,12055,7248,5282,4148,4117
7,시흥시,512586,22919,26079,24244,26331,33929,38031,38115,47483,44519,48922,47875,41941,28947,15384,10361,8146,9360
8,군포시,283322,10380,12611,12798,15309,18301,20716,19712,22083,21292,25942,25122,25094,20017,11209,8137,6462,8137
9,의왕시,162992,6047,7117,6745,8399,11215,11791,10506,12111,11314,14062,14788,15316,12814,7199,5011,3809,4748


In [8]:
inf5 = pd.read_excel("01_시군구별 제1권-서울 인천 경기 강원_2019년.xlsx",sheet_name='98p심평원')


# 1. 3번 행을 칼럼명으로 변경
inf5.columns = inf5.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf5 = inf5.drop(index=[0, 1, 2, 3, 4])

# 3. 46, 47, 48번 행 삭제
inf5 = inf5.drop(index=[54, 55, 56])

# 인덱스 리셋
inf5 = inf5.reset_index(drop=True)


inf5.columns = [col.split('\n')[0] for col in inf5.columns]

inf5 = inf5.drop(columns = [ "조  산  원", "Division"])

# 창원시 관련 행 필터링 및 합계 계산
suwon_gu = ["수원시장안구           ", "수원시권선구           ", "수원시팔달구           ", "수원시영통구           "]
suwon_data = inf5[inf5['구      분'].isin(suwon_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
suwon_row = pd.DataFrame([['수원시'] + suwon_data.tolist()], columns=inf5.columns)

# 창원시 관련 행 필터링 및 합계 계산
sungnam_gu = ["성남시수정구           ", "성남시중원구           ", "성남시분당구           "]
sungnam_data = inf5[inf5['구      분'].isin(sungnam_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
sungnam_row = pd.DataFrame([['성남시'] + sungnam_data.tolist()], columns=inf5.columns)

# 창원시 관련 행 필터링 및 합계 계산
anyang_gu = ["안양시만안구           ", "안양시동안구           "]
anyang_data = inf5[inf5['구      분'].isin(anyang_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
anyang_row = pd.DataFrame([['안양시'] + anyang_data.tolist()], columns=inf5.columns)

# 창원시 행 추가 및 기존 구 삭제
inf5 = pd.concat([inf5[~inf5['구      분'].isin(suwon_gu)], suwon_row], ignore_index=True)
inf5 = pd.concat([inf5[~inf5['구      분'].isin(sungnam_gu)], sungnam_row], ignore_index=True)
inf5 = pd.concat([inf5[~inf5['구      분'].isin(anyang_gu)], anyang_row], ignore_index=True)


# 창원시 관련 구 이름
changwon_gu = ['보  건  소', '보건지소','보건진료소', '보건의료원']

# 예시: 'col1', 'col2', 'col3'의 값을 합쳐 'new_col' 생성
inf5['보건소'] = inf5[['보  건  소', '보건지소','보건진료소', '보건의료원']].sum(axis=1)

inf5 = inf5.drop(columns = [  '보  건  소', '보건지소','보건진료소', '보건의료원'])

inf5 = inf5.rename(columns={'구      분': '지역'})

inf5['지역'] = inf5['지역'] + ' ' + inf5.groupby('지역').cumcount().replace(0, '').astype(str)


inf5

,지역,계,상급종합병원,종합병원,병 원,요양병원,의 원,치과병원,치과의원,한방병원,한 의 원,약 국,보건소
0,서울,22739,13,44,228,124,8628,65,4836,53,3617,5095,33
1,종로구,668,2,2,3,0,193,1,163,0,131,172,1
2,중구,728,0,2,4,0,241,6,211,1,95,167,1
3,용산구,435,0,1,2,0,135,0,106,0,62,127,1
4,성동구,593,1,0,6,3,213,1,127,0,93,148,1
5,광진구,723,1,1,5,3,268,2,154,0,118,170,1
6,동대문구,832,1,3,14,9,260,4,157,1,157,224,1
7,중랑구,689,0,3,12,3,237,0,148,0,108,177,1
8,성북구,729,1,0,6,7,251,1,149,0,113,200,1
9,강북구,628,0,1,8,6,225,1,124,0,96,166,1


In [9]:
inf6 = pd.read_excel("01_시군구별 제1권-서울 인천 경기 강원_2019년.xlsx",sheet_name='100p')


# 1. 3번 행을 칼럼명으로 변경
inf6.columns = inf6.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf6 = inf6.drop(index=[0, 1, 2, 3, 4])

# 3. 46, 47, 48번 행 삭제
inf6 = inf6.drop(index=[55, 56, 57])

# 인덱스 리셋
inf6 = inf6.reset_index(drop=True)


inf6.columns = [col.split('\n')[0] for col in inf6.columns]

inf6 = inf6.drop(columns = [ "조  산  원", "Division"])

# 창원시 관련 행 필터링 및 합계 계산
ansan_gu = ["안산시상록구           ", "안산시단원구           "]
ansan_data = inf6[inf6['구      분'].isin(ansan_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
ansan_row = pd.DataFrame([['안산시'] + ansan_data.tolist()], columns=inf6.columns)

# 창원시 관련 행 필터링 및 합계 계산
goyang_gu = ["고양시덕양구           ", "고양시일산동구", "고양시일산서구"]
goyang_data = inf6[inf6['구      분'].isin(goyang_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
goyang_row = pd.DataFrame([['고양시'] + goyang_data.tolist()], columns=inf6.columns)

# 창원시 관련 행 필터링 및 합계 계산
yongin_gu = ["용인시처인구", "용인시기흥구", "용인시수지구"]
yongin_data = inf6[inf6['구      분'].isin(yongin_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
yongin_row = pd.DataFrame([['용인시'] + yongin_data.tolist()], columns=inf6.columns)

# 창원시 행 추가 및 기존 구 삭제
inf6 = pd.concat([inf6[~inf6['구      분'].isin(ansan_gu)], ansan_row], ignore_index=True)
inf6 = pd.concat([inf6[~inf6['구      분'].isin(goyang_gu)], goyang_row], ignore_index=True)
inf6 = pd.concat([inf6[~inf6['구      분'].isin(yongin_gu)], yongin_row], ignore_index=True)
inf6 = inf6.rename(columns={'구      분': '지역'})



# 창원시 관련 구 이름
changwon_gu = ['보  건  소', '보건지소','보건진료소', '보건의료원']

# 예시: 'col1', 'col2', 'col3'의 값을 합쳐 'new_col' 생성
inf6['보건소'] = inf6[['보  건  소', '보건지소','보건진료소', '보건의료원']].sum(axis=1)

inf6 = inf6.drop(columns = [  '보  건  소', '보건지소','보건진료소', '보건의료원'])

inf6 = inf6.rename(columns={'구      분': '지역'})

inf6['지역'] = inf6['지역'] + ' ' + inf6.groupby('지역').cumcount().replace(0, '').astype(str)


inf6

,지역,계,상급종합병원,종합병원,병 원,요양병원,의 원,치과병원,치과의원,한방병원,한 의 원,약 국,보건소
0,광명시,530,0,1,5,5,205,0,114,0,81,118,1
1,평택시,765,0,4,11,7,253,0,148,1,111,207,22
2,동두천시,140,0,0,1,10,41,0,26,0,18,43,1
3,과천시,92,0,0,0,NaN,38,1,17,0,14,21,1
4,구리시,418,0,1,9,5,146,0,93,0,66,95,3
5,남양주시,912,0,3,12,19,308,0,194,0,139,225,11
6,오산시,314,0,2,6,6,103,1,70,1,41,83,1
7,시흥시,622,0,3,4,13,226,2,131,9,74,154,6
8,군포시,429,0,2,3,8,140,2,97,0,68,107,2
9,의왕시,189,0,0,2,4,62,1,44,0,30,43,2


In [10]:
inf7 = pd.read_excel("01_시군구별 제1권-서울 인천 경기 강원_2019년.xlsx",sheet_name='102p')


# 1. 3번 행을 칼럼명으로 변경
inf7.columns = inf7.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf7 = inf7.drop(index=[0, 1, 2, 3, 4, 5])

# 3. 46, 47, 48번 행 삭제
inf7 = inf7.drop(index=[55, 56, 57])

# 인덱스 리셋
inf7 = inf7.reset_index(drop=True)
inf7 = inf7.loc[:, ~inf7.columns.isna()]

inf7.columns = [col.split('\n')[0] for col in inf7.columns]
inf7.columns = [col.split('physical')[0] for col in inf7.columns]
inf7 = inf7.drop(columns = [ "Division"])

# 창원시 관련 행 필터링 및 합계 계산
suwon_gu = ["수원시장안구           ", "수원시권선구           ", "수원시팔달구           ", "수원시영통구           "]
suwon_data = inf7[inf7['구   분'].isin(suwon_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
suwon_row = pd.DataFrame([['수원시'] + suwon_data.tolist()], columns=inf7.columns)

# 창원시 관련 행 필터링 및 합계 계산
sungnam_gu = ["성남시수정구           ", "성남시중원구           ", "성남시분당구           "]
sungnam_data = inf7[inf7['구   분'].isin(sungnam_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
sungnam_row = pd.DataFrame([['성남시'] + sungnam_data.tolist()], columns=inf7.columns)

# 창원시 관련 행 필터링 및 합계 계산
anyang_gu = ["안양시만안구           ", "안양시동안구           "]
anyang_data = inf7[inf7['구   분'].isin(anyang_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
anyang_row = pd.DataFrame([['안양시'] + anyang_data.tolist()], columns=inf7.columns)

# 창원시 행 추가 및 기존 구 삭제
inf7 = pd.concat([inf7[~inf7['구   분'].isin(suwon_gu)], suwon_row], ignore_index=True)
inf7 = pd.concat([inf7[~inf7['구   분'].isin(sungnam_gu)], sungnam_row], ignore_index=True)
inf7 = pd.concat([inf7[~inf7['구   분'].isin(anyang_gu)], anyang_row], ignore_index=True)




inf7 = inf7.rename(columns={'구   분': '지역'})

inf7['지역'] = inf7['지역'] + ' ' + inf7.groupby('지역').cumcount().replace(0, '').astype(str)

inf7

,지역,계,의 사,치 과 의 사,한 의 사,간호사,약 사,물리치료사,작업치료사,사회복지사
0,서울,114861,30359,7308,5018,52727,9385,8240,1269,555
1,종로구,7447,2300,439,168,3732,495,230,58,25
2,중구,2865,816,275,131,1150,277,202,2,12
3,용산구,2041,574,138,75,957,182,100,8,7
4,성동구,2646,734,167,110,1169,249,185,23,9
5,광진구,3183,927,195,142,1363,298,209,23,26
6,동대문구,4864,1133,369,319,2121,369,419,100,34
7,중랑구,3024,666,190,141,1305,287,346,56,33
8,성북구,3662,951,214,146,1648,330,284,72,17
9,강북구,1797,428,164,120,475,243,292,58,17


In [11]:
inf8 = pd.read_excel("01_시군구별 제1권-서울 인천 경기 강원_2019년.xlsx",sheet_name='104p')


# 1. 3번 행을 칼럼명으로 변경
inf8.columns = inf8.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf8 = inf8.drop(index=[0, 1, 2, 3, 4, 5])

# 3. 46, 47, 48번 행 삭제
inf8 = inf8.drop(index=[56, 57, 58])

# 인덱스 리셋
inf8 = inf8.reset_index(drop=True)
inf8 = inf8.loc[:, ~inf8.columns.isna()]

inf8.columns = [col.split('\n')[0] for col in inf8.columns]
inf8.columns = [col.split('physical')[0] for col in inf8.columns]
inf8 = inf8.drop(columns = [ "Division"])

# 창원시 관련 행 필터링 및 합계 계산
ansan_gu = ["안산시상록구           ", "안산시단원구           "]
ansan_data = inf8[inf8['구   분'].isin(ansan_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
ansan_row = pd.DataFrame([['안산시'] + ansan_data.tolist()], columns=inf8.columns)

# 창원시 관련 행 필터링 및 합계 계산
goyang_gu = ["고양시덕양구           ", "고양시일산동구", "고양시일산서구"]
goyang_data = inf8[inf8['구   분'].isin(goyang_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
goyang_row = pd.DataFrame([['고양시'] + goyang_data.tolist()], columns=inf8.columns)

# 창원시 관련 행 필터링 및 합계 계산
yongin_gu = ["용인시처인구", "용인시기흥구", "용인시수지구"]
yongin_data = inf8[inf8['구   분'].isin(yongin_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
yongin_row = pd.DataFrame([['용인시'] + yongin_data.tolist()], columns=inf8.columns)

# 창원시 행 추가 및 기존 구 삭제
inf8 = pd.concat([inf8[~inf8['구   분'].isin(ansan_gu)], ansan_row], ignore_index=True)
inf8 = pd.concat([inf8[~inf8['구   분'].isin(goyang_gu)], goyang_row], ignore_index=True)
inf8 = pd.concat([inf8[~inf8['구   분'].isin(yongin_gu)], yongin_row], ignore_index=True)
inf8 = inf8.rename(columns={'구   분': '지역'})



inf8 = inf8.rename(columns={'구   분': '지역'})

inf8['지역'] = inf8['지역'] + ' ' + inf8.groupby('지역').cumcount().replace(0, '').astype(str)

inf8

,지역,계,의 사,치 과 의 사,한 의 사,간호사,약 사,물리치료사,작업치료사,사회복지사
0,광명시,1778,469,157,103,565,202,236,35,11
1,평택시,2655,625,203,144,1020,323,296,26,18
2,동두천시,598,124,35,35,237,71,61,17,18
3,과천시,170,48,26,18,17,33,28,0,0
4,구리시,2125,500,117,80,939,177,246,52,14
5,남양주시,2743,660,238,183,885,361,336,43,37
6,오산시,1288,274,99,61,531,137,136,36,14
7,시흥시,2225,459,167,130,830,269,302,47,21
8,군포시,1710,398,136,90,698,187,157,25,19
9,의왕시,571,125,56,36,203,68,54,3,26


In [12]:
inf9 = pd.read_excel("01_시군구별 제1권-서울 인천 경기 강원_2019년.xlsx",sheet_name='106p')


# 1. 3번 행을 칼럼명으로 변경
inf9.columns = inf9.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf9 = inf9.drop(index=[0, 1, 2, 3, 4, 5, 6])

# 3. 46, 47, 48번 행 삭제
inf9 = inf9.drop(index=[56, 57, 58])

# 인덱스 리셋
inf9 = inf9.reset_index(drop=True)

inf9.iloc[:, [3, 4]] = inf9.iloc[:, [4, 3]].values
inf9.iloc[:, [15, 16]] = inf9.iloc[:, [16, 15]].values

inf9 = inf9.loc[:, ~inf9.columns.isna()]

inf9.columns = [col.split('\n')[0] for col in inf9.columns]
inf9.columns = [col.split(' Inpatient room')[0] for col in inf9.columns]
inf9.columns = [col.split('Intensive Care Unit')[0] for col in inf9.columns]
inf9 = inf9.drop(columns = [ '계                                          Total',  '정신과폐쇄',
'격리병실                           Isolation room', '무균치료실                   Aseptic room', "Division"])

# 창원시 관련 행 필터링 및 합계 계산
suwon_gu = ["수원시장안구           ", "수원시권선구           ", "수원시팔달구           ", "수원시영통구           "]
suwon_data = inf9[inf9['구   분'].isin(suwon_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
suwon_row = pd.DataFrame([['수원시'] + suwon_data.tolist()], columns=inf9.columns)

# 창원시 관련 행 필터링 및 합계 계산
sungnam_gu = ["성남시수정구           ", "성남시중원구           ", "성남시분당구           "]
sungnam_data = inf9[inf9['구   분'].isin(sungnam_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
sungnam_row = pd.DataFrame([['성남시'] + sungnam_data.tolist()], columns=inf9.columns)

# 창원시 관련 행 필터링 및 합계 계산
anyang_gu = ["안양시만안구           ", "안양시동안구           "]
anyang_data = inf9[inf9['구   분'].isin(anyang_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
anyang_row = pd.DataFrame([['안양시'] + anyang_data.tolist()], columns=inf9.columns)

# 창원시 행 추가 및 기존 구 삭제
inf9 = pd.concat([inf9[~inf9['구   분'].isin(suwon_gu)], suwon_row], ignore_index=True)
inf9 = pd.concat([inf9[~inf9['구   분'].isin(sungnam_gu)], sungnam_row], ignore_index=True)
inf9 = pd.concat([inf9[~inf9['구   분'].isin(anyang_gu)], anyang_row], ignore_index=True)


inf9 = inf9.rename(columns={'구   분': '지역'})

inf9['지역'] = inf9['지역'] + ' ' + inf9.groupby('지역').cumcount().replace(0, '').astype(str)
inf9.at[39, '지역'] = '부천시'
inf9

,지역,일반입원실,중환자실
0,서울,81044,242
1,종로구,3074,11
2,중구,1227,5
3,용산구,853,2
4,성동구,1790,4
5,광진구,2310,4
6,동대문구,4841,8
7,중랑구,2691,6
8,성북구,3361,17
9,강북구,2207,2


In [13]:
inf10 = pd.read_excel("01_시군구별 제1권-서울 인천 경기 강원_2019년.xlsx",sheet_name='108p')


# 1. 3번 행을 칼럼명으로 변경
inf10.columns = inf10.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf10 = inf10.drop(index=[0, 1, 2, 3, 4, 5, 6])

# 3. 46, 47, 48번 행 삭제
inf10 = inf10.drop(index=[57, 58, 59])

# 인덱스 리셋
inf10 = inf10.reset_index(drop=True)

inf10.iloc[:, [3, 4]] = inf10.iloc[:, [4, 3]].values
inf10.iloc[:, [15, 16]] = inf10.iloc[:, [16, 15]].values

inf10 = inf10.loc[:, ~inf10.columns.isna()]

inf10.columns = [col.split('\n')[0] for col in inf10.columns]
inf10.columns = [col.split(' Inpatient room')[0] for col in inf10.columns]
inf10.columns = [col.split('Intensive Care Unit')[0] for col in inf10.columns]
inf10 = inf10.drop(columns = [ '계                                          Total',  '정신과폐쇄',
'격리병실                           Isolation room', '무균치료실                   Aseptic room', "Division"])

# 창원시 관련 행 필터링 및 합계 계산
ansan_gu = ["안산시상록구           ", "안산시단원구           "]
ansan_data = inf10[inf10['구   분'].isin(ansan_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
ansan_row = pd.DataFrame([['안산시'] + ansan_data.tolist()], columns=inf10.columns)

# 창원시 관련 행 필터링 및 합계 계산
goyang_gu = ["고양시덕양구           ", "고양시일산동구", "고양시일산서구"]
goyang_data = inf10[inf10['구   분'].isin(goyang_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
goyang_row = pd.DataFrame([['고양시'] + goyang_data.tolist()], columns=inf10.columns)

# 창원시 관련 행 필터링 및 합계 계산
yongin_gu = ["용인시처인구", "용인시기흥구", "용인시수지구"]
yongin_data = inf10[inf10['구   분'].isin(yongin_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
yongin_row = pd.DataFrame([['용인시'] + yongin_data.tolist()], columns=inf10.columns)

# 창원시 행 추가 및 기존 구 삭제
inf10 = pd.concat([inf10[~inf10['구   분'].isin(ansan_gu)], ansan_row], ignore_index=True)
inf10 = pd.concat([inf10[~inf10['구   분'].isin(goyang_gu)], goyang_row], ignore_index=True)
inf10 = pd.concat([inf10[~inf10['구   분'].isin(yongin_gu)], yongin_row], ignore_index=True)
inf10 = inf10.rename(columns={'구   분': '지역'})

inf10 = inf10.rename(columns={'구   분': '지역'})

inf10

,지역,일반입원실,중환자실
0,광명시,2099,2
1,평택시,3676,4
2,동두천시,2141,0
3,과천시,0,0
4,구리시,2416,4
5,남양주시,5400,5
6,오산시,2170,2
7,시흥시,4316,3
8,군포시,2141,2
9,의왕시,994,0


In [14]:
inf11 = pd.read_excel("01_시군구별 제1권-서울 인천 경기 강원_2019년.xlsx",sheet_name='110p')


# 1. 3번 행을 칼럼명으로 변경
inf11.columns = inf11.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf11 = inf11.drop(index=[0, 1, 2, 3, 4])

# 3. 46, 47, 48번 행 삭제
inf11 = inf11.drop(index=[54, 55, 56])

# 인덱스 리셋
inf11 = inf11.reset_index(drop=True)


inf11.columns = [col.split('\n')[0] for col in inf11.columns]

inf11 = inf11.drop(columns = [  '계', '유         방',
       '양전자 방사                                 단층 촬영                            PET',
       '체외충격파쇄석기     Extracorporead', '혈관조영장치', '혈관조영장치', 'Division'])

# 창원시 관련 행 필터링 및 합계 계산
suwon_gu = ["수원시장안구           ", "수원시권선구           ", "수원시팔달구           ", "수원시영통구           "]
suwon_data = inf11[inf11['구   분'].isin(suwon_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
suwon_row = pd.DataFrame([['수원시'] + suwon_data.tolist()], columns=inf11.columns)

# 창원시 관련 행 필터링 및 합계 계산
sungnam_gu = ["성남시수정구           ", "성남시중원구           ", "성남시분당구           "]
sungnam_data = inf11[inf11['구   분'].isin(sungnam_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
sungnam_row = pd.DataFrame([['성남시'] + sungnam_data.tolist()], columns=inf11.columns)

# 창원시 관련 행 필터링 및 합계 계산
anyang_gu = ["안양시만안구           ", "안양시동안구           "]
anyang_data = inf11[inf11['구   분'].isin(anyang_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
anyang_row = pd.DataFrame([['안양시'] + anyang_data.tolist()], columns=inf11.columns)

# 창원시 행 추가 및 기존 구 삭제
inf11 = pd.concat([inf11[~inf11['구   분'].isin(suwon_gu)], suwon_row], ignore_index=True)
inf11 = pd.concat([inf11[~inf11['구   분'].isin(sungnam_gu)], sungnam_row], ignore_index=True)
inf11 = pd.concat([inf11[~inf11['구   분'].isin(anyang_gu)], anyang_row], ignore_index=True)


inf11 = inf11.rename(columns={'구   분': '지역'})

inf11['지역'] = inf11['지역'] + ' ' + inf11.groupby('지역').cumcount().replace(0, '').astype(str)


inf11

,지역,자기공명영상기,전산화단층촬영장치,초음파
0,서울,411,418,8337
1,종로구,25,27,404
2,중구,14,18,331
3,용산구,3,6,136
4,성동구,6,9,173
5,광진구,10,10,252
6,동대문구,15,18,336
7,중랑구,14,11,240
8,성북구,11,12,267
9,강북구,3,9,223


In [15]:
inf12 = pd.read_excel("01_시군구별 제1권-서울 인천 경기 강원_2019년.xlsx",sheet_name='112p')


# 1. 3번 행을 칼럼명으로 변경
inf12.columns = inf12.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf12 = inf12.drop(index=[0, 1, 2, 3, 4])

#3. 46, 47, 48번 행 삭제
inf12 = inf12.drop(index=[55, 56, 57])

# 인덱스 리셋
inf12 = inf12.reset_index(drop=True)

# inf12.iloc[:, [3, 4]] = inf12.iloc[:, [4, 3]].values
# inf12.iloc[:, [15, 16]] = inf12.iloc[:, [16, 15]].values

# inf12 = inf12.loc[:, ~inf12.columns.isna()]

inf12.columns = [col.split('\n')[0] for col in inf12.columns]
inf12.columns = [col.split(' Inpatient room')[0] for col in inf12.columns]
inf12.columns = [col.split('Intensive Care Unit')[0] for col in inf12.columns]
inf12 = inf12.drop(columns = [  '계', '유         방',
       '양전자 방사                                 단층 촬영                            PET',
       '체외충격파쇄석기     Extracorporead', '혈관조영장치', '혈관조영장치', 'Division'])

# 창원시 관련 행 필터링 및 합계 계산
ansan_gu = ["안산시상록구           ", "안산시단원구           "]
ansan_data = inf12[inf12['구   분'].isin(ansan_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
ansan_row = pd.DataFrame([['안산시'] + ansan_data.tolist()], columns=inf12.columns)

# 창원시 관련 행 필터링 및 합계 계산
goyang_gu = ["고양시덕양구           ", "고양시일산동구", "고양시일산서구"]
goyang_data = inf12[inf12['구   분'].isin(goyang_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
goyang_row = pd.DataFrame([['고양시'] + goyang_data.tolist()], columns=inf12.columns)

# 창원시 관련 행 필터링 및 합계 계산
yongin_gu = ["용인시처인구", "용인시기흥구", "용인시수지구"]
yongin_data = inf12[inf12['구   분'].isin(yongin_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
yongin_row = pd.DataFrame([['용인시'] + yongin_data.tolist()], columns=inf12.columns)

# 창원시 행 추가 및 기존 구 삭제
inf12 = pd.concat([inf12[~inf12['구   분'].isin(ansan_gu)], ansan_row], ignore_index=True)
inf12 = pd.concat([inf12[~inf12['구   분'].isin(goyang_gu)], goyang_row], ignore_index=True)
inf12 = pd.concat([inf12[~inf12['구   분'].isin(yongin_gu)], yongin_row], ignore_index=True)

inf12 = inf12.rename(columns={'구   분': '지역'})

inf12

,지역,자기공명영상기,전산화단층촬영장치,초음파
0,광명시,8,6,173
1,평택시,15,16,254
2,동두천시,1,1,40
3,과천시,NaN,NaN,17
4,구리시,10,11,146
5,남양주시,11,12,281
6,오산시,5,8,80
7,시흥시,3,12,182
8,군포시,3,8,126
9,의왕시,1,2,41


In [16]:
inf13 = pd.read_excel("01_시군구별 제1권-서울 인천 경기 강원_2019년.xlsx",sheet_name='152p')


# 1. 3번 행을 칼럼명으로 변경
inf13.columns = inf13.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf13 = inf13.drop(index=[0, 1, 2, 3, 4, 5, 6, 7])

# 3. 46, 47, 48번 행 삭제
inf13 = inf13.drop(index=[57, 58])

# 인덱스 리셋
inf13 = inf13.reset_index(drop=True)

# # inf13.iloc[:, [3, 4]] = inf13.iloc[:, [4, 3]].values
# # inf13.iloc[:, [15, 16]] = inf13.iloc[:, [16, 15]].values

inf13 = inf13.loc[:, ~inf13.columns.isna()]

inf13.columns = [col.split('\n')[0] for col in inf13.columns]
# # inf13.columns = [col.split(' Inpatient room')[0] for col in inf13.columns]
# # inf13.columns = [col.split('Intensive Care Unit')[0] for col in inf13.columns]
inf13 = inf13.drop(columns = ['내원일수', '급여일수',
       '진료실인원당     Per Patient', '내원1일당  Per Visit ',
       '급여1일당  Per reimbursed day', 'Division'])
# 창원시 관련 행 필터링 및 합계 계산
suwon_gu = ["수원시장안구           ", "수원시권선구           ", "수원시팔달구           ", "수원시영통구           "]
suwon_data = inf13[inf13['구 분'].isin(suwon_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
suwon_row = pd.DataFrame([['수원시'] + suwon_data.tolist()], columns=inf13.columns)

# 창원시 관련 행 필터링 및 합계 계산
sungnam_gu = ["성남시수정구           ", "성남시중원구           ", "성남시분당구           "]
sungnam_data = inf13[inf13['구 분'].isin(sungnam_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
sungnam_row = pd.DataFrame([['성남시'] + sungnam_data.tolist()], columns=inf13.columns)

# 창원시 관련 행 필터링 및 합계 계산
anyang_gu = ["안양시만안구           ", "안양시동안구           "]
anyang_data = inf13[inf13['구 분'].isin(anyang_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
anyang_row = pd.DataFrame([['안양시'] + anyang_data.tolist()], columns=inf13.columns)

# 창원시 행 추가 및 기존 구 삭제
inf13 = pd.concat([inf13[~inf13['구 분'].isin(suwon_gu)], suwon_row], ignore_index=True)
inf13 = pd.concat([inf13[~inf13['구 분'].isin(sungnam_gu)], sungnam_row], ignore_index=True)
inf13 = pd.concat([inf13[~inf13['구 분'].isin(anyang_gu)], anyang_row], ignore_index=True)

inf13 = inf13.rename(columns={'구 분': '지역'})

inf13['지역'] = inf13['지역'] + ' ' + inf13.groupby('지역').cumcount().replace(0, '').astype(str)

inf13

,지역,진료실인원,진료비,급여비
0,서울,9222102,16491896666.670002,12671033234.134001
1,종로구,142653,266543452.46,204204820.554
2,중구,126376,245686701.86,189825410.62
3,용산구,211036,392772740.66,302346923.449
4,성동구,284607,515085748.24,393257352.028
5,광진구,333399,561764311.07,426215430.415
6,동대문구,324160,636834175.25,492132942.716
7,중랑구,370955,745503423.83,581012959.061
8,성북구,413433,750184705.28,574847079.444
9,강북구,291334,599855657.65,470220533.227


In [17]:
inf14 = pd.read_excel("01_시군구별 제1권-서울 인천 경기 강원_2019년.xlsx",sheet_name='154p')


# 1. 3번 행을 칼럼명으로 변경
inf14.columns = inf14.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf14 = inf14.drop(index=[0, 1, 2, 3, 4, 5, 6, 7])

# 3. 46, 47, 48번 행 삭제
inf14 = inf14.drop(index=[58, 59])

# 인덱스 리셋
inf14 = inf14.reset_index(drop=True)

# # inf14.iloc[:, [3, 4]] = inf14.iloc[:, [4, 3]].values
# # inf14.iloc[:, [15, 16]] = inf14.iloc[:, [16, 15]].values

inf14 = inf14.loc[:, ~inf14.columns.isna()]

inf14.columns = [col.split('\n')[0] for col in inf14.columns]
# # inf14.columns = [col.split(' Inpatient room')[0] for col in inf14.columns]
# # inf14.columns = [col.split('Intensive Care Unit')[0] for col in inf14.columns]
inf14 = inf14.drop(columns = ['내원일수', '급여일수',
       '진료실인원당     Per Patient', '내원1일당  Per Visit ',
       '급여1일당  Per reimbursed day', 'Division'])
# 창원시 관련 행 필터링 및 합계 계산
ansan_gu = ["안산시상록구           ", "안산시단원구           "]
ansan_data = inf14[inf14['구 분'].isin(ansan_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
ansan_row = pd.DataFrame([['안산시'] + ansan_data.tolist()], columns=inf14.columns)

# 창원시 관련 행 필터링 및 합계 계산
goyang_gu = ["고양시덕양구           ", "고양시일산동구", "고양시일산서구"]
goyang_data = inf14[inf14['구 분'].isin(goyang_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
goyang_row = pd.DataFrame([['고양시'] + goyang_data.tolist()], columns=inf14.columns)

# 창원시 관련 행 필터링 및 합계 계산
yongin_gu = ["용인시처인구", "용인시기흥구", "용인시수지구"]
yongin_data = inf14[inf14['구 분'].isin(yongin_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
yongin_row = pd.DataFrame([['용인시'] + yongin_data.tolist()], columns=inf14.columns)

# 창원시 행 추가 및 기존 구 삭제
inf14 = pd.concat([inf14[~inf14['구 분'].isin(ansan_gu)], ansan_row], ignore_index=True)
inf14 = pd.concat([inf14[~inf14['구 분'].isin(goyang_gu)], goyang_row], ignore_index=True)
inf14 = pd.concat([inf14[~inf14['구 분'].isin(yongin_gu)], yongin_row], ignore_index=True)

inf14 = inf14.rename(columns={'구 분': '지역'})

inf14

,지역,진료실인원,진료비,급여비
0,광명시,303221,532296803.96,409189487.088
1,평택시,498633,839193611.77,644738057.364
2,동두천시,88740,187702827.34,148269161.311
3,과천시,54998,85542960.55,64449405.16
4,구리시,188219,333746389.32,257431225.131
5,남양주시,665616,1159722682.45,892468379.909
6,오산시,221518,340057132.03,260943074.435
7,시흥시,475785,749469555.35,573456649.614
8,군포시,265926,443833026.21,340189821.027
9,의왕시,153213,254489457.77,192992569.9


In [18]:
inf15 = pd.read_excel("01_시군구별 제1권-서울 인천 경기 강원_2019년.xlsx",sheet_name='170p관내')


# 1. 3번 행을 칼럼명으로 변경
inf15.columns = inf15.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf15 = inf15.drop(index=[0, 1, 2, 3, 4, 5, 6, 7])

# 3. 46, 47, 48번 행 삭제
inf15 = inf15.drop(index=[57, 58])

# 인덱스 리셋
inf15 = inf15.reset_index(drop=True)

# # inf15.iloc[:, [3, 4]] = inf15.iloc[:, [4, 3]].values
# # inf15.iloc[:, [15, 16]] = inf15.iloc[:, [16, 15]].values

inf15 = inf15.loc[:, ~inf15.columns.isna()]

inf15.columns = [col.split('\n')[0] for col in inf15.columns]
# # inf15.columns = [col.split(' Inpatient room')[0] for col in inf15.columns]
# # inf15.columns = [col.split('Intensive Care Unit')[0] for col in inf15.columns]
inf15 = inf15.drop(columns = ['내원일수', '급여일수',
       '진료실인원당     Per Patient', '내원1일당  Per Visit ',
       '급여1일당  Per reimbursed day', 'Division'])
# 창원시 관련 행 필터링 및 합계 계산
suwon_gu = ["수원시장안구           ", "수원시권선구           ", "수원시팔달구           ", "수원시영통구           "]
suwon_data = inf15[inf15['구 분'].isin(suwon_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
suwon_row = pd.DataFrame([['수원시'] + suwon_data.tolist()], columns=inf15.columns)

# 창원시 관련 행 필터링 및 합계 계산
sungnam_gu = ["성남시수정구           ", "성남시중원구           ", "성남시분당구           "]
sungnam_data = inf15[inf15['구 분'].isin(sungnam_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
sungnam_row = pd.DataFrame([['성남시'] + sungnam_data.tolist()], columns=inf15.columns)

# 창원시 관련 행 필터링 및 합계 계산
anyang_gu = ["안양시만안구           ", "안양시동안구           "]
anyang_data = inf15[inf15['구 분'].isin(anyang_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
anyang_row = pd.DataFrame([['안양시'] + anyang_data.tolist()], columns=inf15.columns)

# 창원시 행 추가 및 기존 구 삭제
inf15 = pd.concat([inf15[~inf15['구 분'].isin(suwon_gu)], suwon_row], ignore_index=True)
inf15 = pd.concat([inf15[~inf15['구 분'].isin(sungnam_gu)], sungnam_row], ignore_index=True)
inf15 = pd.concat([inf15[~inf15['구 분'].isin(anyang_gu)], anyang_row], ignore_index=True)

inf15 = inf15.rename(columns={'구 분': '지역'})

inf15['지역'] = inf15['지역'] + ' ' + inf15.groupby('지역').cumcount().replace(0, '').astype(str)

inf15

,지역,진료실인원,진료비,급여비
0,서울,8900597,14770542863.389999,11330096455.271
1,종로구,110332,105064467.81,78723899.69
2,중구,95889,86846605.12,66446736.618
3,용산구,153877,122070527.3,93964772.935
4,성동구,225667,192344535.7,144448850.07
5,광진구,285249,292387825.68,219875007.489
6,동대문구,272227,305205868.9,233208867.458
7,중랑구,316256,325205352.55,251061846.667
8,성북구,337071,326876053.64,248884861.082
9,강북구,247625,216668287.03,166590781.25


In [19]:
inf16 = pd.read_excel("01_시군구별 제1권-서울 인천 경기 강원_2019년.xlsx",sheet_name='172p')


# 1. 3번 행을 칼럼명으로 변경
inf16.columns = inf16.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf16 = inf16.drop(index=[0, 1, 2, 3, 4, 5, 6, 7])

# 3. 46, 47, 48번 행 삭제
inf16 = inf16.drop(index=[58, 59])

# 인덱스 리셋
inf16 = inf16.reset_index(drop=True)

# # inf16.iloc[:, [3, 4]] = inf16.iloc[:, [4, 3]].values
# # inf16.iloc[:, [15, 16]] = inf16.iloc[:, [16, 15]].values

inf16 = inf16.loc[:, ~inf16.columns.isna()]

inf16.columns = [col.split('\n')[0] for col in inf16.columns]
# # inf16.columns = [col.split(' Inpatient room')[0] for col in inf16.columns]
# # inf16.columns = [col.split('Intensive Care Unit')[0] for col in inf16.columns]
inf16 = inf16.drop(columns = ['내원일수', '급여일수',
       '진료실인원당     Per Patient', '내원1일당  Per Visit ',
       '급여1일당  Per reimbursed day', 'Division'])
# 창원시 관련 행 필터링 및 합계 계산
ansan_gu = ["안산시상록구           ", "안산시단원구           "]
ansan_data = inf16[inf16['구 분'].isin(ansan_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
ansan_row = pd.DataFrame([['안산시'] + ansan_data.tolist()], columns=inf16.columns)

# 창원시 관련 행 필터링 및 합계 계산
goyang_gu = ["고양시덕양구           ", "고양시일산동구", "고양시일산서구"]
goyang_data = inf16[inf16['구 분'].isin(goyang_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
goyang_row = pd.DataFrame([['고양시'] + goyang_data.tolist()], columns=inf16.columns)

# 창원시 관련 행 필터링 및 합계 계산
yongin_gu = ["용인시처인구", "용인시기흥구", "용인시수지구"]
yongin_data = inf16[inf16['구 분'].isin(yongin_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
yongin_row = pd.DataFrame([['용인시'] + yongin_data.tolist()], columns=inf16.columns)

# 창원시 행 추가 및 기존 구 삭제
inf16 = pd.concat([inf16[~inf16['구 분'].isin(ansan_gu)], ansan_row], ignore_index=True)
inf16 = pd.concat([inf16[~inf16['구 분'].isin(goyang_gu)], goyang_row], ignore_index=True)
inf16 = pd.concat([inf16[~inf16['구 분'].isin(yongin_gu)], yongin_row], ignore_index=True)

inf16 = inf16.rename(columns={'구 분': '지역'})

inf16

,지역,진료실인원,진료비,급여비
0,광명시,268878,247157497.14,186617420.664
1,평택시,450810,478775938.2,359287860.406
2,동두천시,78887,85669400.62,67082720.51
3,과천시,42066,19640048.2,14489049.724
4,구리시,164934,174319875.13,133587582.643
5,남양주시,576001,478662819.67,361714556.198
6,오산시,191473,160191099.66,119790118.78
7,시흥시,413081,316919489.5,235850636.943
8,군포시,234306,213571888.49,160949719.811
9,의왕시,110121,57658664.5,43030407.531


In [20]:
inf17 = pd.read_excel("01_시군구별 제1권-서울 인천 경기 강원_2019년.xlsx",sheet_name='182p관외')


# 1. 3번 행을 칼럼명으로 변경
inf17.columns = inf17.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf17 = inf17.drop(index=[0, 1, 2, 3, 4, 5, 6, 7])

# 3. 46, 47, 48번 행 삭제
inf17 = inf17.drop(index=[57, 58])

# 인덱스 리셋
inf17 = inf17.reset_index(drop=True)

# # inf17.iloc[:, [3, 4]] = inf17.iloc[:, [4, 3]].values
# # inf17.iloc[:, [15, 16]] = inf17.iloc[:, [16, 15]].values

inf17 = inf17.loc[:, ~inf17.columns.isna()]

inf17.columns = [col.split('\n')[0] for col in inf17.columns]
# # inf17.columns = [col.split(' Inpatient room')[0] for col in inf17.columns]
# # inf17.columns = [col.split('Intensive Care Unit')[0] for col in inf17.columns]
inf17 = inf17.drop(columns = ['내원일수', '급여일수',
       '진료실인원당     Per Patient', '내원1일당  Per Visit ',
       '급여1일당  Per reimbursed day', 'Division'])
# 창원시 관련 행 필터링 및 합계 계산
suwon_gu = ["수원시장안구           ", "수원시권선구           ", "수원시팔달구           ", "수원시영통구           "]
suwon_data = inf17[inf17['구 분'].isin(suwon_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
suwon_row = pd.DataFrame([['수원시'] + suwon_data.tolist()], columns=inf17.columns)

# 창원시 관련 행 필터링 및 합계 계산
sungnam_gu = ["성남시수정구           ", "성남시중원구           ", "성남시분당구           "]
sungnam_data = inf17[inf17['구 분'].isin(sungnam_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
sungnam_row = pd.DataFrame([['성남시'] + sungnam_data.tolist()], columns=inf17.columns)

# 창원시 관련 행 필터링 및 합계 계산
anyang_gu = ["안양시만안구           ", "안양시동안구           "]
anyang_data = inf17[inf17['구 분'].isin(anyang_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
anyang_row = pd.DataFrame([['안양시'] + anyang_data.tolist()], columns=inf17.columns)

# 창원시 행 추가 및 기존 구 삭제
inf17 = pd.concat([inf17[~inf17['구 분'].isin(suwon_gu)], suwon_row], ignore_index=True)
inf17 = pd.concat([inf17[~inf17['구 분'].isin(sungnam_gu)], sungnam_row], ignore_index=True)
inf17 = pd.concat([inf17[~inf17['구 분'].isin(anyang_gu)], anyang_row], ignore_index=True)

inf17 = inf17.rename(columns={'구 분': '지역'})

inf17['지역'] = inf17['지역'] + ' ' + inf17.groupby('지역').cumcount().replace(0, '').astype(str)

inf17

,지역,진료실인원,진료비,급여비
0,서울,2234352,1721353803.28,1340936778.863
1,종로구,115899,161478984.65,125480920.864
2,중구,101385,158840096.74,123378674.002
3,용산구,178027,270702213.36,208382150.514
4,성동구,225037,322741212.54,248808501.958
5,광진구,232427,269376485.39,206340422.926
6,동대문구,233633,331628306.35,258924075.258
7,중랑구,267662,420298071.28,329951112.394
8,성북구,319457,423308651.64,325962218.362
9,강북구,219072,383187370.62,303629751.977


In [21]:
inf18 = pd.read_excel("01_시군구별 제1권-서울 인천 경기 강원_2019년.xlsx",sheet_name='184p')


# 1. 3번 행을 칼럼명으로 변경
inf18.columns = inf18.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf18 = inf18.drop(index=[0, 1, 2, 3, 4, 5, 6, 7])

# 3. 46, 47, 48번 행 삭제
inf18 = inf18.drop(index=[58, 59])

# 인덱스 리셋
inf18 = inf18.reset_index(drop=True)

# # inf18.iloc[:, [3, 4]] = inf18.iloc[:, [4, 3]].values
# # inf18.iloc[:, [15, 16]] = inf18.iloc[:, [16, 15]].values

inf18 = inf18.loc[:, ~inf18.columns.isna()]

inf18.columns = [col.split('\n')[0] for col in inf18.columns]
# # inf18.columns = [col.split(' Inpatient room')[0] for col in inf18.columns]
# # inf18.columns = [col.split('Intensive Care Unit')[0] for col in inf18.columns]
inf18 = inf18.drop(columns = ['내원일수', '급여일수',
       '진료실인원당     Per Patient', '내원1일당  Per Visit ',
       '급여1일당  Per reimbursed day', 'Division'])
# 창원시 관련 행 필터링 및 합계 계산
ansan_gu = ["안산시상록구           ", "안산시단원구           "]
ansan_data = inf18[inf18['구 분'].isin(ansan_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
ansan_row = pd.DataFrame([['안산시'] + ansan_data.tolist()], columns=inf18.columns)

# 창원시 관련 행 필터링 및 합계 계산
goyang_gu = ["고양시덕양구           ", "고양시일산동구", "고양시일산서구"]
goyang_data = inf18[inf18['구 분'].isin(goyang_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
goyang_row = pd.DataFrame([['고양시'] + goyang_data.tolist()], columns=inf18.columns)

# 창원시 관련 행 필터링 및 합계 계산
yongin_gu = ["용인시처인구", "용인시기흥구", "용인시수지구"]
yongin_data = inf18[inf18['구 분'].isin(yongin_gu)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
yongin_row = pd.DataFrame([['용인시'] + yongin_data.tolist()], columns=inf18.columns)

# 창원시 행 추가 및 기존 구 삭제
inf18 = pd.concat([inf18[~inf18['구 분'].isin(ansan_gu)], ansan_row], ignore_index=True)
inf18 = pd.concat([inf18[~inf18['구 분'].isin(goyang_gu)], goyang_row], ignore_index=True)
inf18 = pd.concat([inf18[~inf18['구 분'].isin(yongin_gu)], yongin_row], ignore_index=True)

inf18 = inf18.rename(columns={'구 분': '지역'})

inf18['지역'] = inf18['지역'] + ' ' + inf18.groupby('지역').cumcount().replace(0, '').astype(str)

inf18

,지역,진료실인원,진료비,급여비
0,광명시,196518,285139306.82,222572066.424
1,평택시,256728,360417673.57,285450196.958
2,동두천시,58915,102033426.72,81186440.801
3,과천시,45762,65902912.35,49960355.436
4,구리시,123131,159426514.19,123843642.488
5,남양주시,450811,681059862.78,530753823.711
6,오산시,138314,179866032.37,141152955.655
7,시흥시,304778,432550065.85,337606012.671
8,군포시,175532,230261137.72,179240101.216
9,의왕시,135346,196830793.27,149962162.369


In [22]:

# merged_df = pd.merge(inf, inf2, on = '구 분')

# merged_df

result1 = pd.concat([inf, inf2], axis=0)
result2 = pd.concat([inf3, inf4], axis=0)
result3 = pd.concat([inf5, inf6], axis=0)
result4 = pd.concat([inf7, inf8], axis=0)
result5 = pd.concat([inf9, inf10], axis=0)
result6 = pd.concat([inf11, inf12], axis=0)
result7 = pd.concat([inf13, inf14], axis=0)
result8 = pd.concat([inf15, inf16], axis=0)
result9 = pd.concat([inf17, inf18], axis=0)


col = '지역'
result1[col] = result1[col].str.strip()
result2[col] = result2[col].str.strip()
result3[col] = result3[col].str.strip()
result4[col] = result4[col].str.strip()
result5[col] = result5[col].str.strip()
result6[col] = result6[col].str.strip()
result7[col] = result7[col].str.strip()
result8[col] = result8[col].str.strip()
result9[col] = result9[col].str.strip()

result1 = result1.reset_index(drop=True)
result2 = result2.reset_index(drop=True)
result3 = result3.reset_index(drop=True)
result4 = result4.reset_index(drop=True)
result5 = result5.reset_index(drop=True)
result6 = result6.reset_index(drop=True)
result7 = result7.reset_index(drop=True)
result8 = result8.reset_index(drop=True)
result9 = result9.reset_index(drop=True)




In [23]:
result1['지역'] = result1['지역'] + ' ' + result1.groupby('지역').cumcount().replace(0, '').astype(str)
result2['지역'] = result2['지역'] + ' ' + result1.groupby('지역').cumcount().replace(0, '').astype(str)
result3['지역'] = result3['지역'] + ' ' + result1.groupby('지역').cumcount().replace(0, '').astype(str)
result4['지역'] = result4['지역'] + ' ' + result1.groupby('지역').cumcount().replace(0, '').astype(str)
result5['지역'] = result5['지역'] + ' ' + result1.groupby('지역').cumcount().replace(0, '').astype(str)
result6['지역'] = result6['지역'] + ' ' + result1.groupby('지역').cumcount().replace(0, '').astype(str)
result7['지역'] = result7['지역'] + ' ' + result1.groupby('지역').cumcount().replace(0, '').astype(str)
result8['지역'] = result8['지역'] + ' ' + result1.groupby('지역').cumcount().replace(0, '').astype(str)
result9['지역'] = result9['지역'] + ' ' + result1.groupby('지역').cumcount().replace(0, '').astype(str)

In [24]:


dataframes = [result1, result2, result3, result4, result5, result6, result7, result8, result9]

result = dataframes[0]
for df in dataframes[1:]:
    result = result.merge(df, on='지역')
result = result.drop_duplicates(subset='지역', keep='first')
result

,지역,의료보장,근 로 자,공ㆍ교,지 역,의료급여 1종,의료급여 2종,계_x,0-4세,5-9세,...,초음파,진료실인원_x,진료비_x,급여비_x,진료실인원_y,진료비_y,급여비_y,진료실인원,진료비,급여비
0,서울,9933336,6410576,576211,2695880,179257,71412,9933336,307840,366494,...,8337,9222102,16491896666.670002,12671033234.134001,8900597,14770542863.389999,11330096455.271,2234352,1721353803.28,1340936778.863
1,종로구,157094,92512,11069,49231,3442,840,157094,3727,4828,...,404,142653,266543452.46,204204820.554,110332,105064467.81,78723899.69,115899,161478984.65,125480920.864
2,중구,138071,84314,5503,44532,2967,755,138071,4191,4012,...,331,126376,245686701.86,189825410.62,95889,86846605.12,66446736.618,101385,158840096.74,123378674.002
3,용산구,231921,143113,14686,68331,4416,1375,231921,7071,7480,...,136,211036,392772740.66,302346923.449,153877,122070527.3,93964772.935,178027,270702213.36,208382150.514
4,성동구,305714,201231,14947,82873,4507,2156,305714,11377,10926,...,173,284607,515085748.24,393257352.028,225667,192344535.7,144448850.07,225037,322741212.54,248808501.958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,고성군,27618,12376,4396,9658,957,231,27618,608,743,...,9,25913,61013776.98,47959273.09,15626,9447675.09,7199235.815,23393,51566101.89,40760037.275
84,양양군,27933,13124,3601,9875,1140,193,27933,587,800,...,2,26369,58176902.1,45769744.964,17996,9049425.65,6903335.143,23451,49127476.45,38866409.821
85,안산시,710915,453186,20942,219595,12491,4701,710915,22634,27632,...,401,654085,1093116107.92,843251490.866,556869,543282456.8,412995320.046,427596,549833651.12,430256170.82
86,고양시,1078782,714375,70096,275339,14331,4641,1078782,37294,47924,...,644,1007361,1708082989.66,1311119655.598,845026,774375174.55,587510473.901,737388,933707815.11,723609181.697


In [25]:
file_path = "2019 경기 강원.csv"  # 저장할 파일 이름과 경로 설정
result.to_csv(file_path, index=False, encoding='utf-8-sig')  # UTF-8 인코딩으로 저장